In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
import scipy


import util_diagnostic
import util_visualization
import util_preprocessing
import util_calculation
import util_modelling

def reload():
    importlib.reload(util_diagnostic)
    importlib.reload(util_visualization)
    importlib.reload(util_preprocessing)
    importlib.reload(util_calculation)
    importlib.reload(util_modelling)

pd.set_option('display.max_columns', None)

file_name = "Data - Final.csv"
file_name_test="Data - Test.csv"
# datatype={'Nama':object,'Gender':object,'Bahasa':float,'Inggris':float,'TIK':float,'Matematika':float,'Fisika':float,'Kimia':float,'Geografi':float,'Sosiologi':float,'Antropologi':float,'Tgl_Daftar_Kuliah':object,'Lama_Kuliah':object}
df = pd.read_csv(file_name,delimiter=';')
df_test=pd.read_csv(file_name_test,delimiter=';')
df=df.iloc[:,1:]
columns = df.columns
test_columns=df_test.columns
display(df.head())
df.shape

ImportError: cannot import name 'DataError' from 'pandas.core.base' (d:\tools\miniconda\envs\lombaDAC\lib\site-packages\pandas\core\base.py)

## Change Data Type

In [ ]:
df.info()

In [ ]:
num_col=['Bahasa','Inggris','TIK','Matematika','Fisika','Kimia','Biologi','Geografi','Sosiologi','Antropologi']
df=util_preprocessing.changeToFloatNums(df,num_col,np.nan)
df['Tgl_Daftar_Kuliah']=df['Tgl_Daftar_Kuliah'].astype(object)

In [ ]:
df.info()

## CheckNUll

In [ ]:
util_preprocessing.checkNullDataFrame(df)

## Fill Missing

In [ ]:
reload()
util_visualization.drawCountPlot(df, "Tinggal_Dengan", "Tinggal_Dengan",dropna=False,title='countplot Tinggal_Dengan')

In [ ]:
util_visualization.drawCountPlot(df, "Biaya", "Biaya",dropna=False, percentage=False,title='countplot Biaya')

In [ ]:
# karna mirip sebarannya NaN kita isi dengan Nan

df['Tinggal_Dengan'].fillna('Kosong',inplace=True)
df['Biaya'].fillna('Kosong',inplace=True)


util_preprocessing.checkNullDataFrame(df)

In [ ]:
reload()
util_calculation.calculateCorrelationNumeric(df)


In [ ]:
reload()
util_calculation.calculateCorrelationMix(df)

In [ ]:
df[num_col].describe()

In [ ]:
# lebih aman kalo di drop
df.dropna(inplace=True)
util_preprocessing.checkNullDataFrame(df)

## FE

In [5]:
df

NameError: name 'df' is not defined

## Encode

In [ ]:
df

In [ ]:
cat_col=util_preprocessing.getOnlyColmnsWithType(df, 'object')
num_col=util_preprocessing.getOnlyColmnsWithType(df, 'number')

encodedDf,scaler=util_preprocessing.normalizeNumericColumn(df,num_col)
encodedDf,labelEncoder=util_preprocessing.encodeCategoricalColumn(df,cat_col)
encodedDf

In [ ]:
x=encodedDf[test_columns]
y=encodedDf['Fakultas']

In [ ]:
model=util_modelling.prepareKNN()
model


In [ ]:
x

In [ ]:
reload()
util_modelling.getStratifedKFoldScore([model],x.values,y.values)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
import sklearn

parameters_KNN = {
    'n_neighbors': (1,100, 5),
    'leaf_size': (20,60,5),
    'p': (1,2),
    'weights': ('uniform', 'distance'),
    'metric': ('minkowski', 'chebyshev'),
}



model=sklearn.model_selection.GridSearchCV(KNeighborsClassifier(),param_grid=parameters_KNN,scoring = 'accuracy', n_jobs = -1,cv = 10)

In [ ]:
model.fit(x.values,y.values)

In [ ]:
model.best_params_ 

In [ ]:
model.best_score_

In [ ]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=.5,random_state =123)
clf = LazyClassifier(verbose=0,ignore_warnings=True, custom_metric=None)
models,predictions = clf.fit(X_train, X_test, y_train, y_test)
models

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold
from tpot import TPOTClassifier

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

x=df[test_columns].values
y=df['Fakultas'].values

model = TPOTClassifier(generations=5, population_size=50, cv=cv, scoring='accuracy', verbosity=2, random_state=1, n_jobs=-1)
model.fit(x, y)
model.export('tpot_sonar_best_model.py')